In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
from urllib.parse import quote

In [17]:
def scrape_jobs(position,location):
    position = quote(position, safe='-')
    location = quote(location, safe='-')
    page = 1
    data = []
    while True:
        template = 'https://www.jobstreet.co.id/id/{}-jobs/in-{}?page={}'
        url = template.format(position,location,page)
        
        html_text = requests.get(url).text
        soup = BeautifulSoup(html_text, 'lxml')
        jobs = soup.find_all('div',class_='y735df0 _1akoxc50 _1akoxc54')
        print('jobs found', len(jobs),'url',url)
        zeroResult = soup.find('section',class_='y735df0 y735df1 _1iz8dgs82 _1iz8dgs6y _1iz8dgsaa _1iz8dgs96')
        if zeroResult:
            print('No more jobs found')
            break
        page += 1

        for job in jobs:
            try:
                job_role = job.find('h3').text.strip()
            except AttributeError:
                job_role = ''
            try:
                company_name = job.find('a',class_='y735df0 y735dff  y735df0 y735dff _842p0a0 _842p0a1').text.strip()
                # print(company_name)
            except AttributeError:
                company_name = ''
            try:
                location_company = job.find('a',class_='y735df0 _1iz8dgs4y _94v4w0 _94v4w1 _94v4w21 _1wzghjf4 _94v4w7').text.strip()
                # print(location_company)
            except AttributeError:
                location_company = ''
            try:
                posted_date = job.find('span',class_='y735df0 _1iz8dgs4y _94v4w0 _94v4w1 _94v4w22 _1wzghjf4 _94v4w7').span.text.strip()
                # print(posted_date)
            except AttributeError:
                posted_date = ''
            try:
                skill = (job.find('a',class_='y735df0 _1iz8dgsr _1iz8dgsf6 _1iz8dgsbu _1iz8dgs4y _1iz8dgsfm').text).split('subClassification:')[-1].strip()
                print(skill)
            except AttributeError:
                skill = ''
            
            # try:
            #     more_detail_link = job.header.h2.a['href']
            # except AttributeError:
            #     more_detail_link = ''
            
            extract_date = datetime.today().strftime('%Y-%m-%d')
            
            data.append({'Role':job_role, 'Company Name':company_name, 'Location':location_company, 'Posted Date':posted_date,'Extracted Date':extract_date,'Key Skill': skill})
    df = pd.DataFrame(data)
    
    df.to_csv(f'list_of_{position}_jobstreet.csv', index=False, encoding='utf-8')

        

In [18]:
if __name__ == '__main__':
    position = 'data analyst'
    location = 'jakarta'

    scrape_jobs(position,location)

jobs found 94 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=1
jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=2
jobs found 104 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=3
jobs found 101 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=4
jobs found 100 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=5
jobs found 101 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=6
jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=7
jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=8
jobs found 104 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=9
jobs found 103 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=10
jobs found 102 url https://www.jobstreet.co.id/id/data%20analyst-jobs/in-jakarta?page=11
jobs found 104 url https://www.

# Jobstreet

In [2]:
jobstreet_base_url ='https://www.jobstreet.co.id'

In [3]:
r = requests.get('https://www.jobstreet.co.id/id/jobs')
soup = BeautifulSoup(r.content, 'lxml')
jobs = soup.find_all('div',class_='y735df0 _1akoxc50 _1akoxc54')

In [11]:
job_url = []

In [9]:
for job in jobs:
    for job_card in job.find_all('div', class_='y735df0 _1iz8dgs6m'):
        job_card_details = job_card.find_all('div', class_='y735df0')
        for job_card_detail in job_card_details:
            article = job_card_detail.find('article', class_='y735df0 y735df1 _1iz8dgs7i _1iz8dgs6e _1iz8dgs9q _1iz8dgs8m _1iz8dgs66 _1iz8dgsh _1iz8dgs5e uo6mkb _1iz8dgs4i _1iz8dgs4b _94v4w18 _94v4w1b _1iz8dgs32 _1iz8dgs35')
            if article:
                link = article.find('a', href=True)
                if link:
                    job_url.append(jobstreet_base_url + link['href'])
                    # print(link['href'])
# print(len(job_url))

96


In [ ]:
for job in jobs:
    for job_card in job.find_all('div', data-='y735df0 _1iz8dgs6m'):
        job_card_details = job_card.find_all('div', class_='y735df0')
        for job_card_detail in job_card_details:
            article = job_card_detail.find('article', class_='y735df0 y735df1 _1iz8dgs7i _1iz8dgs6e _1iz8dgs9q _1iz8dgs8m _1iz8dgs66 _1iz8dgsh _1iz8dgs5e uo6mkb _1iz8dgs4i _1iz8dgs4b _94v4w18 _94v4w1b _1iz8dgs32 _1iz8dgs35')
            if article:
                link = article.find('a', href=True)
                if link:
                    job_url.append(jobstreet_base_url + link['href'])
                    # print(link['href'])
# print(len(job_url))

In [8]:
job_url

['https://www.jobstreet.co.id/id/job/74062398?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74051176?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74044358?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74061571?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74029326?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74027446?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/73997854?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74025053?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74027452?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74048719?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/73971287?type=standout&ref=search-standalone',
 'https://www.jobstreet.co.id/id/job/74043861?type=standout&ref=search-stand

# Detail About Jobs

In [12]:
job_url = 'https://www.jobstreet.co.id/id/job/74044358?type=standout&ref=search-standalone'

In [16]:
role_title = soup.find_all('div',class_='y735df0 _1iz8dgs70')
role_title

[]

# Clean Code 

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
from urllib.parse import quote

In [86]:
search = 'data scientist'
location = 'surabaya'
search = search.replace(' ','-')
location = location.replace(' ','-')

In [83]:
def find_tag_value(soup, tag, attribute):
    try:
        return soup.find(tag, attrs={'data-automation':attribute}).text.strip()
    except AttributeError:
        return None

In [105]:
page = 1
data = []
while True:
    r = requests.get('https://www.jobstreet.co.id/id/{}-jobs/in-{}?page={}'.format(search, location, page))
    soup = BeautifulSoup(r.content, 'lxml')
    job_cards = soup.find_all('article',attrs={'data-card-type':'JobCard'})
    if len(job_cards) == 0:
        break
    print('page',page,'jobs found', len(job_cards))

    for job_card in job_cards:
    
        job_title = find_tag_value(job_card, 'a', 'jobTitle')
        
        company_name = find_tag_value(job_card, 'a', 'jobCompany')
        
        location_city = find_tag_value(job_card, 'a', 'jobLocation')
        
        salary = find_tag_value(job_card, 'span', 'jobSalary')
        if salary:
            salary = salary.replace(u'\xa0', u'')

        job_classification = find_tag_value(job_card, 'a', 'jobClassification')
        
        job_sub_classification = find_tag_value(job_card, 'a', 'jobSubClassification')
        
        job_desc = find_tag_value(job_card, 'span', 'jobShortDescription')
        
        facility = job_card.find('ul',class_='y735df0 y735df3 _1akoxc50 _1akoxc54')
        if facility is not None:
            facility = facility.find_all('li')
            facility_list = [item.text.strip() for item in facility]
            # make into 1 text with comma delimiter
            facility_list = ', '.join(facility_list)

        else:
            facility_list = ''

        posted_date = find_tag_value(job_card, 'span', 'jobListingDate')

        job_id = job_card['data-job-id']
        
        # job_detail_request = requests.get('https://www.jobstreet.co.id/id/{}-jobs/in-{}?jobId={}&type=standout'.format(search, location, job_id))
        # soup_job_detail_request = BeautifulSoup(job_detail_request.content, 'lxml')
        # job_cards = soup_job_detail_request.find_all('article',attrs={'data-card-type':'JobCard'})

        data.append({'Job Title':job_title, 'Company Name':company_name, 'Location':location_city, 'Salary':salary, 'Job Classification':job_classification, 'Job Sub Classification':job_sub_classification, 'Job Description':job_desc, 'Facility':facility_list, 'Posted Date':posted_date})
        
    page += 1

df = pd.DataFrame(data)
df.to_csv('jobstreet.csv', index=False, encoding='utf-8')

    

page 1 jobs found 8
73920644
73802735
73428408
73464299
73386115
73494339
73464293
74058511
